In [57]:
import json
import numpy as np
import math
import os

In [58]:
# game_file_name = 'pure_strategy_game.json'
game_file_name = 'mixed_strategy_game.json'

with open(game_file_name, 'r') as f:
    game = np.array(json.load(f)['game_matrix'])
    M, N = len(game), len(game[0])

print(f'Таблица игры:')
print(game)

Таблица игры:
[[-2 -2  0]
 [-2  0 -1]
 [ 0 -1 -1]]


In [59]:
def simplify_game(game):
    def simplify_game_rows(game, lte):
        dominated_is = set()
        for dominant_i in range(0, M):
            for dominated_i in range(0, M):
                if dominant_i != dominated_i and all(lte(game[dominated_i], game[dominant_i])):
                    dominated_is.add(dominated_i)

        return dominated_is
        
    dominated_is = list(simplify_game_rows(game, lte=np.less_equal))
    dominated_js = list(simplify_game_rows(game.T, lte=np.greater_equal))

    game = np.delete(game, dominated_is, axis=0)
    game = np.delete(game, dominated_js, axis=1)

    return game

game = simplify_game(game)
M, N = len(game), len(game[0])

print('Упрощенная таблица игры')
print(game)

Упрощенная таблица игры
[[-2 -2  0]
 [-2  0 -1]
 [ 0 -1 -1]]


In [60]:
def find_pure_strategy_equilibrium(game):
    row_mins = np.min(game, axis=1)
    maximin_i = np.argmax(row_mins)
    maximin = row_mins[maximin_i]

    column_maxs = np.max(game, axis=0)
    minimax_j = np.argmin(column_maxs)
    minimax = column_maxs[minimax_j]

    if maximin == minimax:
        return maximin_i, minimax_j
    
    return None

print('Решение в чистых стратегиях')
print(find_pure_strategy_equilibrium(game))

Решение в чистых стратегиях
None


In [61]:
def find_mixed_strategy_equilibrium(game): 

    def run_simplex(goal='max'):
        lin_system = {
            'f': [1] * N,
            'goal': goal,
            'constraints': [
                {
                    "coefs": list([int(x) for x in game[i]]),
                    "type": "lte" if goal == 'max' else 'gte',
                    "b": 1
                } for i in range(M)
            ]
        }

        with open(f'lin_system_{goal}.json', 'w+') as f:
            json.dump(lin_system, f)

        exit_code = os.system('py simplex.py -f lin_system_max.json')
        if exit_code != 0:
            print(f'gateway error: simplex exit code {exit_code}')
            return None

        with open('simplex_answer.json', 'r') as f:
            answer = json.load(f)['answer']

        return answer
    
    if game.min() < 0:
        added_value = abs(game.min())+1
        print('Неотрицательная матрица игры')
        game = game + added_value
        print(game, end='\n\n')

    q = run_simplex(goal='max')
    x = np.array(q) / sum(q)
    y = x
    v = (1 / sum(q)) - added_value

    return x, y, v

x, y, v = find_mixed_strategy_equilibrium(game)
print(f'x: {x}')
print(f'y = {y}')
print(f'v = {v}')

Неотрицательная матрица игры
[[1 1 3]
 [1 3 2]
 [3 2 2]]

x: [0.14285714 0.28571429 0.57142857]
y = [0.14285714 0.28571429 0.57142857]
v = -0.8571428571428572
